In [499]:
import pandas as pd
names = (pd.read_csv('Indian_Names.csv')['Name']).dropna(axis=0).to_list()
import re
# Regular expression pattern to match any string that contains special characters, digits, or spaces
pattern = re.compile(r'[^a-zA-Z]')
print(names.__len__())
# Filter out strings containing special characters, digits, or spaces
names = [name for name in names if not pattern.search(name)]
names.__len__()

6485


6466

In [500]:
s= '..'.join(names)
pairs = sorted(set(list(ch1+ch2 for ch1,ch2 in zip(s,s[1:]))))

chars = sorted(list(set(''.join(s))))
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

stoipair={}
for ch1 in stoi:
    for ch2 in stoi:
        stoipair[ch1+ch2] = stoi[ch1]*27 +stoi[ch2]
itospair={i:s for i,s in enumerate(stoipair)}

{'..': 0, '.a': 1, '.b': 2, '.c': 3, '.d': 4, '.e': 5, '.f': 6, '.g': 7, '.h': 8, '.i': 9, '.j': 10, '.k': 11, '.l': 12, '.m': 13, '.n': 14, '.o': 15, '.p': 16, '.q': 17, '.r': 18, '.s': 19, '.t': 20, '.u': 21, '.v': 22, '.w': 23, '.x': 24, '.y': 25, '.z': 26, 'a.': 27, 'aa': 28, 'ab': 29, 'ac': 30, 'ad': 31, 'ae': 32, 'af': 33, 'ag': 34, 'ah': 35, 'ai': 36, 'aj': 37, 'ak': 38, 'al': 39, 'am': 40, 'an': 41, 'ao': 42, 'ap': 43, 'aq': 44, 'ar': 45, 'as': 46, 'at': 47, 'au': 48, 'av': 49, 'aw': 50, 'ax': 51, 'ay': 52, 'az': 53, 'b.': 54, 'ba': 55, 'bb': 56, 'bc': 57, 'bd': 58, 'be': 59, 'bf': 60, 'bg': 61, 'bh': 62, 'bi': 63, 'bj': 64, 'bk': 65, 'bl': 66, 'bm': 67, 'bn': 68, 'bo': 69, 'bp': 70, 'bq': 71, 'br': 72, 'bs': 73, 'bt': 74, 'bu': 75, 'bv': 76, 'bw': 77, 'bx': 78, 'by': 79, 'bz': 80, 'c.': 81, 'ca': 82, 'cb': 83, 'cc': 84, 'cd': 85, 'ce': 86, 'cf': 87, 'cg': 88, 'ch': 89, 'ci': 90, 'cj': 91, 'ck': 92, 'cl': 93, 'cm': 94, 'cn': 95, 'co': 96, 'cp': 97, 'cq': 98, 'cr': 99, 'cs': 100

In [501]:
import torch 
N = torch.zeros(729,27, dtype= torch.int32)


In [503]:
N = torch.zeros(729,27, dtype= torch.int32)
for name in names:
    ch = list('..'+name+'.')
    for ch1,ch2,ch3 in zip(ch,ch[1:],ch[2:]):
            pair = ch1+ch2
            ixpair = stoipair[pair]
            ix = stoi[ch3]
            N[ixpair][ix] += 1

In [504]:
a = N.sum(1,keepdims=True)
prob =N.float()/ N.sum(1,keepdims=True)

In [505]:
g = torch.Generator().manual_seed(2147483650)
for i in range(30):
  
  out = ['..']
  ix=0
  while True:
    ixpair = stoipair[''.join(out)[-2:]]
    p = prob[ixpair]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

..hai.
..munimalindhara.
..ba.
..jagat.
..chibhivya.
..hena.
..anshwan.
..derparmesh.
..sugawatiwanj.
..kum.
..sabhetabramalmijeeman.
..yaz.
..jen.
..mehmmeetili.
..jeer.
..sandrahranji.
..krukireer.
..kratn.
..sim.
..murbiyashilam.
..dil.
..hali.
..bharmam.
..kumohati.
..ka.
..helat.
..ma.
..nishahir.
..sankumanank.
..sinir.


In [506]:
# create the training set of bigrams (x,y)
xs_train, ys_train = [], []

for w in names:
  chs = list('..'+w+'.')
  for ch1, ch2,ch3 in zip(chs, chs[1:],chs[2:]):
    pair=ch1+ch2
    ixpair=stoipair[pair]
    ix=stoi[ch3]
    xs_train.append(ixpair)
    ys_train.append(ix)
    
xs_train = torch.tensor(xs_train)
ys_train = torch.tensor(ys_train)

In [509]:
xs_train.__len__(),ys_train.__len__()

(47561, 47561)

In [510]:
import torch.nn.functional as f
xenc_train = f.one_hot(xs_train,num_classes=729).float()

In [511]:
g = torch.Generator().manual_seed(2147483647)
w=torch.randn(729,27,generator=g,requires_grad=True)

In [512]:
logitsN = xenc_train @ N.float()
countsN = logitsN
probN = countsN / countsN.sum(1,keepdims=True)
lossN= -probN[torch.arange(xs_train.__len__()), ys_train].log().mean()
lossN.item()


2.062042474746704

In [513]:
logits_train = xenc_train @ w
counts_train = logits_train.exp()
prob_train = counts_train / counts_train.sum(1,keepdims=True)
loss= -prob_train[torch.arange(xs_train.__len__()), ys_train].log().mean()
loss


tensor(3.8565, grad_fn=<NegBackward0>)

In [514]:
w.grad = None
loss.backward()
w.data+= -80* w.grad

In [519]:
for i in range(100):
    logits_train = xenc_train @ w
    counts_train = logits_train.exp()
    prob_train = counts_train / counts_train.sum(1,keepdims=True)
    loss= -prob_train[torch.arange(xs_train.__len__()), ys_train].log().mean() + 0.01*(w**2).mean()
    print(loss.item())

    w.grad = None
    loss.backward()
    w.data+= -10* w.grad

2.1291675567626953
2.1268534660339355
2.1250529289245605
2.123697280883789
2.1227076053619385
2.122006416320801
2.1215217113494873
2.121194362640381
2.120976686477661
2.1208338737487793
2.1207408905029297
2.120680093765259
2.120640277862549
2.1206135749816895
2.1205949783325195
2.12058162689209
2.1205716133117676
2.1205637454986572
2.1205568313598633
2.1205503940582275
2.120544672012329
2.1205391883850098
2.1205341815948486
2.1205286979675293
2.1205239295959473
2.120518207550049
2.120513439178467
2.1205084323883057
2.1205034255981445
2.1204986572265625
2.1204936504364014
2.120488405227661
2.1204833984375
2.120478391647339
2.120473623275757
2.1204686164855957
2.1204636096954346
2.1204586029052734
2.1204535961151123
2.1204488277435303
2.1204440593719482
2.120438814163208
2.120434045791626
2.120429039001465
2.1204240322113037
2.1204192638397217
2.1204142570495605
2.1204092502593994
2.1204042434692383
2.1203994750976562
2.120394229888916
2.120389223098755
2.120384693145752
2.12037944793701

In [520]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(50):
  
  out = ['..']
  ix=0
  while True:
    
    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    
    ixpair = stoipair[(''.join(out))[-2:]]
    xenc = f.one_hot(torch.tensor([ixpair]), num_classes=729).float()
    logits = xenc @ w # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    
    ix = torch.multinomial(p, num_samples=1, replacement=True,generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

..junidha.
..anakar.
..presha.
..ambir.
..shin.
..tohima.
..subeetalantauramilendashabinikrwarti.
..sajainay.
..afta.
..farmunthaf.
..dhumerprinsumahrupen.
..coba.
..ya.
..bachar.
..sabir.
..shim.
..in.
..wanit.
..anaasnhavirususushad.
..asfmxtdnail.
..seeparamilat.
..ranrari.
..ish.
..bir.
..rujeerjebahvubhaha.
..van.
..mongrajdkhudditta.
..mal.
..raj.
..laki.
..iq.
..amjpocbzzhqmfali.
..sat.
..fa.
..sal.
..khuvslitikissalaeveer.
..kamewanabushemru.
..aneshwar.
..muramoda.
..dipznojialiypargha.
..tazrwanti.
..kaymheelvin.
..ri.
..himahakesh.
..mil.
..bhashabbin.
..prena.
..preepwkgjwarwffarin.
..shvjvikashannujrajanosan.
..karbhurihandeshdab.


In [ ]:

probs = w.exp() / w.exp().sum(1,keepdims=True)
probs[stoipair['ef']][stoi['q']]

In [ ]:
probs[stoipair['ef']]

In [ ]:
N[stoipair['aw']][stoi['p']]